In [1]:
import pandas as pd
import string

In [2]:
import nltk
from nltk.tokenize import word_tokenize
from nltk.probability import FreqDist
from nltk.corpus import stopwords
from string import punctuation
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.stem.porter import PorterStemmer
from nltk.tokenize import TweetTokenizer

In [3]:
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\chris\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\chris\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\chris\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [4]:
punctuation = []
for i in string.punctuation:
    punctuation.append(str(i))

In [52]:
df = pd.read_excel('../data/qa_data.xlsx')

In [53]:
df.head()

,Subject,Content,Source,Category
0,NaN,Wie werden die Geräte innerhalb meiner Wohnung...,FAQ (Q&A),Glasfaser
1,NaN,Gibt es spezifische Qualitätsmerkmale für Gesc...,FAQ (Q&A),Glasfaser
2,NaN,Welche Angebote können über das Glasfasernetz ...,FAQ (Q&A),Glasfaser
3,NaN,Wieso macht die (rasche) Vertragsunterschrift ...,FAQ (Q&A),Glasfaser
4,NaN,Wie lange dauert die Erschliessung in der Stad...,FAQ (Q&A),Glasfaser


In [54]:
df['Content'] = df['Content'].str.lower()
df['Content'] = df['Content'].str.replace(r'[^\w\s]+', '')

In [55]:
stop_words=set(stopwords.words("german"))

In [56]:
tokens = []
for i in df['Content']:
    i = i.replace('energie wasser bern', '')
    temp = word_tokenize(i)
    tokens.append([word for word in temp if word not in stop_words])
df['Tokens'] = tokens

In [57]:
category_dict = {}
for i in set(df['Category']):
    temp = []
    for j in df.loc[(df['Category']== i)&(df['Source']=='FAQ (Q&A)')]['Tokens']:
        temp.extend(j)
    category_dict[i] = temp

In [60]:
cat_top10 ={}
cat_top10_inc_counts = {}
for i in category_dict.keys():
    fdist = FreqDist(category_dict[i])
    cat_top10_inc_counts[i] = fdist.most_common(10)
    cat_top10[i] = [i[0] for i in fdist.most_common(10)]
pd.DataFrame(cat_top10).to_excel('top_10_words.xls')

In [61]:
cat_top20 ={}
cat_top20_inc_counts = {}
for i in category_dict.keys():
    fdist = FreqDist(category_dict[i])
    cat_top20_inc_counts[i] = fdist.most_common(20)
    cat_top20[i] = [i[0] for i in fdist.most_common(20)]
pd.DataFrame(cat_top20).to_excel('top_20_words.xls')
pd.DataFrame(cat_top20).to_csv('top_20_words.csv')

In [18]:
df_top10 = pd.DataFrame(cat_top10)

In [19]:
df_top10

,Glasfaser,Kehricht,Strom,Internet,Netz,Warme,Mobilitat,Umzug,Diverses,Storungen,Wasser
0,bern,kehricht,ökostrom,router,ewb,erdgas,chf,umzug,wasser,klären,wasser
1,energie,tonne,strom,ewbtv,strom,biogas,erdgas,melden,energie,bitte,schweiz
2,wasser,pro,energie,geräte,kosten,bern,erdgasfahrzeuge,bringt,bern,tel,trinkwasser
3,glasfasernetz,strom,stromprodukt,ewbinternet,bern,energie,gibt,meist,kunden,ausgefallen,liter
4,erschliessung,ab,wasser,kontrollieren,energie,wasser,ebikes,bürokratischen,smart,ab,rund
5,wohnung,müssen,kunden,nutzen,kunden,beim,fahrzeuge,erledigungen,zähler,handelt,bern
6,service,deponiert,produziert,kundendienst,warum,etwa,akku,onlineformulare,strom,nachbarn,mineralwässer
7,eigentümer,kwh,mehr,fritzbox,müssen,pro,schweiz,erleichtern,meter,bern,prozent
8,anschluss,holz,kundinnen,internet,netzanschluss,jahr,ja,abmeldung,teil,störung,hahnenwasser
9,angeschlossen,erzeugt,bern,11,wasser,preise,km,möchte,jahresrechnung,rufen,qualität


In [20]:
df_top10_scores = df_top10.copy()

In [21]:
all_scores = []
max_scores = []
for i in df['Tokens']:
    score = dict.fromkeys(df_top10.columns, 0)
    for j, j_in in zip(df_top10.columns, list(df_top10.index)):
        for l in df_top10[j]:
            if l in i:
                score[j] += 10-j_in
    all_scores.append(score)
    max_scores.append(max(score, key=score.get))
            

In [22]:
df['max_scores'] = max_scores
df['all_scores'] = all_scores

In [23]:
correct = [x == y for x,y in zip(df['Category'], df['max_scores'])]

In [24]:
df['correct'] = correct

In [25]:
sum(df.loc[df['Source']=='FAQ (Q&A)']['correct'])/len(df.loc[df['Source']=='FAQ (Q&A)'])

0.45429362880886426

In [ ]:
import pandas as pd
from nltk.tokenize import word_tokenize
from nltk.probability import FreqDist

from google.cloud import language
from google.cloud.language import enums
from google.cloud.language import types

path = 'google_credentials.json' 
client = language.LanguageServiceClient.from_service_account_json(path)

nltk.download('stopwords')
df_top10 = pd.read_excel('top_10_words.xls')

def preprocess(text):
    text = text.lower()
    text = text.replace(r'[^\w\s]+', '')
    tokens = word_tokenize(text)
    tokens = [word for word in tokens if word not in stop_words]
    return tokens

def get_score(tokens):
    score = dict.fromkeys(df_top10.columns, 0)
    for j, j_in in zip(df_top10.columns, list(df_top10.index)):
        for l in df_top10[j]:
            if l in tokens:
                score[j] += 10-j_in
    category = (max(score, key=score.get))
    return score, category

def google_sentiment(text):
    # Score is positive or negative sentiment
    # Magnitude is how much the text displays that
    document = language.types.Document(
            content=text,
            type=language.enums.Document.Type.PLAIN_TEXT)
    annotations = client.analyze_sentiment(document=document)
    score = annotations.document_sentiment.score
    magnitude = annotations.document_sentiment.magnitude
    if score > 2 & magnitude > 4:
        ex_negative = True
    return ex_negative

def packaged_results(text, user_id, etc):  #user_id, etc just given as examples
    score, category = get_score(preprocess(text))
    ex_negative = google_sentiment(text)
    return {'user_id':user_id, 'text': text, 'category': category, 'score': score, 'ex_negative': ex_negative}

In [26]:
def preprocess(text):
    text = text.lower()
    text = text.replace(r'[^\w\s]+', '')
    tokens = word_tokenize(text)
    tokens = [word for word in tokens if word not in stop_words]
    return tokens

In [27]:
def get_score(tokens):
    score = dict.fromkeys(df_top10.columns, 0)
    for j, j_in in zip(df_top10.columns, list(df_top10.index)):
        for l in df_top10[j]:
            if l in tokens:
                score[j] += 10-j_in
    category = (max(score, key=score.get))
    return score, category

In [28]:
def get_multiple(series):
    categories = []
    scores = []
    for i in series:
        score, category =get_score(preprocess(i))
        scores.append(score)
        categories.append(category)
    return categories, scores

# Testing

In [31]:
df_email = pd.read_excel('../data/Mails.xlsx')

In [32]:
df_email.head()

,Betreff,Mail,Zuständig,Antwort,Kategorie
0,Wechsel Stromprodukt,Sehr geehrte Damen und Herren\n\nGerne möchte ...,Allgemein,Verweis Kundenportal,Strom
1,Re: Aufschaltbestätigung,"Guten Tag\n\nAlles bestens soweit, herzlichen ...",Internet,FAQ,"Internet, TV"
2,Re: Aufschaltbestätigung,Guten Abend \n\nNachdem ich seit dem ersten Ta...,Internet,NaN,"Internet, TV, Störungen"
3,NaN,Sehr geehrte Damen und Herren\n \nLeider sehe ...,Allgemein,NaN,"Strom, Wärme"
4,DRINGEND: Ihre Anfrage betreffend Glasfaserint...,Sehr geehrte Damen und Herren\n\nIch hab’s wir...,Beschwerdemanagement,NaN,"Internet, Glasfaser"


In [33]:
categories, scores = get_multiple(df_email['Mail'])

In [34]:
df_email['categories'] = categories

In [35]:
df_email.to_excel('email_cats.xls')

In [42]:
df_email['Mail'].loc[0]

'Sehr geehrte Damen und Herren\n\nGerne möchte ich ab sofort mein Stromprodukt wechseln von Natur.Strom zu Basis.Strom. Bitte veranlassen Sie dies auf den nächstmöglichen Zeitpunkt. Besten Dank.'

# Sentiment Analysis

In [48]:
from google.cloud import language
from google.cloud.language import enums
from google.cloud.language import types

In [49]:
#Setup

path = '/Users/chris/OneDrive/flatiron/own_work/attest/google_credentials.json' 
client = language.LanguageServiceClient.from_service_account_json(path)

In [50]:
def google_sentiment(input):
    document = language.types.Document(
            content=input,
            type=language.enums.Document.Type.PLAIN_TEXT)
    annotations = client.analyze_sentiment(document=document)
    score = annotations.document_sentiment.score
    magnitude = annotations.document_sentiment.magnitude
    return score, magnitude

In [51]:
google_sentiment(df_email['Mail'].loc[0])

(0.6000000238418579, 2.5)